In [1]:
# Dependencies
import matplotlib.pyplot as plt
%matplotlib inline

import os
import numpy as np
import tensorflow as tf
import pandas as pd

import keras
from keras.preprocessing import image
import operator

C:\Users\OW93790\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
 # Load the model
from keras.models import load_model
model = load_model("models/sklesion.h5")

In [3]:
# Keys for diagnoses in original data and English
dx = ['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc']
diagnoses = ["Bowen's Disease", "Basal Cell Carcinoma", "Benign Keratosis-like Lesion", "Dermatofibroma", "Melanoma", "Melanocytic Nevi", "Vascular Lesions"]
dx_legend = {dx[i]:diagnoses[i] for i in range(7)}
print(dx_legend)

{'akiec': "Bowen's Disease", 'bcc': 'Basal Cell Carcinoma', 'bkl': 'Benign Keratosis-like Lesion', 'df': 'Dermatofibroma', 'mel': 'Melanoma', 'nv': 'Melanocytic Nevi', 'vasc': 'Vascular Lesions'}


In [4]:
# Read in DataFrame of stored images and types
metadata_df=pd.read_csv("Data/HAM10000_metadata.csv")
metadata_df.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization
0,HAM_0000118,ISIC_0027419.jpg,bkl,histo,80.0,male,scalp
1,HAM_0000118,ISIC_0025030.jpg,bkl,histo,80.0,male,scalp
2,HAM_0002730,ISIC_0026769.jpg,bkl,histo,80.0,male,scalp
3,HAM_0002730,ISIC_0025661.jpg,bkl,histo,80.0,male,scalp
4,HAM_0001466,ISIC_0031633.jpg,bkl,histo,75.0,male,ear


In [5]:
 # Default Image Size for sklesion
image_size = (200, 200)

In [6]:
def predict(image_path):
    """Read in image and return unlabeled list of confidence"""
    img = image.load_img(image_path, target_size=image_size)
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    predictions = list(model.predict(x)[0])
    return predictions

In [7]:
def predictdx(predictions):
    """Read in list of confidence and return tuple of diagnosis and confidence"""
    ibest = predictions.index(max(predictions))
    return (dx[ibest], predictions[ibest])

In [8]:
def accu_rate(sampledx, predictions):
    """Evaluate accuracy of prediction against sample"""
    idx = dx.index(sampledx)
    return predictions[idx]

In [9]:
def dxtranslate(sampledx):
    """Translate dx into English"""
    return diagnoses_legend[sampledx]

In [10]:
dx_prediction = []
confidence = []
accuracy = []

for sample in range(10015):
    sampledx = metadata_df["dx"][sample]
    image_path = os.path.join("Images/HAM10000_images", metadata_df["image_id"][sample])
    predictions = predict(image_path)
    bestdx = predictdx(predictions)
    sample_acc = accu_rate(sampledx, predictions)
    if (sample % 10)==0:
        print(f"{sample} samples predicted")
    dx_prediction.append(bestdx[0])
    confidence.append(bestdx[1])
    accuracy.append(sample_acc)

predictions_df = pd.DataFrame({"dx_prediction":dx_prediction, "confidence":confidence, "accuracy":accuracy})

0 samples predicted
10 samples predicted
20 samples predicted
30 samples predicted
40 samples predicted
50 samples predicted
60 samples predicted
70 samples predicted
80 samples predicted
90 samples predicted
100 samples predicted
110 samples predicted
120 samples predicted
130 samples predicted
140 samples predicted
150 samples predicted
160 samples predicted
170 samples predicted
180 samples predicted
190 samples predicted
200 samples predicted
210 samples predicted
220 samples predicted
230 samples predicted
240 samples predicted
250 samples predicted
260 samples predicted
270 samples predicted
280 samples predicted
290 samples predicted
300 samples predicted
310 samples predicted
320 samples predicted
330 samples predicted
340 samples predicted
350 samples predicted
360 samples predicted
370 samples predicted
380 samples predicted
390 samples predicted
400 samples predicted
410 samples predicted
420 samples predicted
430 samples predicted
440 samples predicted
450 samples predicted

3620 samples predicted
3630 samples predicted
3640 samples predicted
3650 samples predicted
3660 samples predicted
3670 samples predicted
3680 samples predicted
3690 samples predicted
3700 samples predicted
3710 samples predicted
3720 samples predicted
3730 samples predicted
3740 samples predicted
3750 samples predicted
3760 samples predicted
3770 samples predicted
3780 samples predicted
3790 samples predicted
3800 samples predicted
3810 samples predicted
3820 samples predicted
3830 samples predicted
3840 samples predicted
3850 samples predicted
3860 samples predicted
3870 samples predicted
3880 samples predicted
3890 samples predicted
3900 samples predicted
3910 samples predicted
3920 samples predicted
3930 samples predicted
3940 samples predicted
3950 samples predicted
3960 samples predicted
3970 samples predicted
3980 samples predicted
3990 samples predicted
4000 samples predicted
4010 samples predicted
4020 samples predicted
4030 samples predicted
4040 samples predicted
4050 sample

7190 samples predicted
7200 samples predicted
7210 samples predicted
7220 samples predicted
7230 samples predicted
7240 samples predicted
7250 samples predicted
7260 samples predicted
7270 samples predicted
7280 samples predicted
7290 samples predicted
7300 samples predicted
7310 samples predicted
7320 samples predicted
7330 samples predicted
7340 samples predicted
7350 samples predicted
7360 samples predicted
7370 samples predicted
7380 samples predicted
7390 samples predicted
7400 samples predicted
7410 samples predicted
7420 samples predicted
7430 samples predicted
7440 samples predicted
7450 samples predicted
7460 samples predicted
7470 samples predicted
7480 samples predicted
7490 samples predicted
7500 samples predicted
7510 samples predicted
7520 samples predicted
7530 samples predicted
7540 samples predicted
7550 samples predicted
7560 samples predicted
7570 samples predicted
7580 samples predicted
7590 samples predicted
7600 samples predicted
7610 samples predicted
7620 sample

In [11]:
metadata_predict_df = metadata_df.merge(predictions_df,left_index=True,right_index=True)
metadata_predict_df.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization,dx_prediction,confidence,accuracy
0,HAM_0000118,ISIC_0027419.jpg,bkl,histo,80.0,male,scalp,nv,1.0,0.0
1,HAM_0000118,ISIC_0025030.jpg,bkl,histo,80.0,male,scalp,nv,1.0,0.0
2,HAM_0002730,ISIC_0026769.jpg,bkl,histo,80.0,male,scalp,nv,1.0,0.0
3,HAM_0002730,ISIC_0025661.jpg,bkl,histo,80.0,male,scalp,nv,1.0,0.0
4,HAM_0001466,ISIC_0031633.jpg,bkl,histo,75.0,male,ear,nv,1.0,0.0


In [12]:
metadata_predict_df.to_csv("Data/metadata_predictions.csv")

In [14]:
overallaccuracy=sum(accuracy)/len(accuracy)
print(overallaccuracy)

0.6684434138104469
